In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn #xây dựng mô hình
from sklearn.utils import resample #tăng giảm các mẫu
from sklearn.preprocessing import StandardScaler #quy đổi về cùng giá trị để so sánh
from sklearn.model_selection import train_test_split #tách dữ liệu

In [2]:
# đọc dữ liệu
df = pd.read_csv('../DuLieuThucHanh/framingham.csv')

In [3]:
# xem số lượng dư liệu
print(df.shape)
# các tên các cột 
print(df.columns)
# xem kiểu dữ liệu các cột
df.dtypes

(4240, 16)
Index(['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')


male                 int64
age                  int64
education          float64
currentSmoker        int64
cigsPerDay         float64
BPMeds             float64
prevalentStroke      int64
prevalentHyp         int64
diabetes             int64
totChol            float64
sysBP              float64
diaBP              float64
BMI                float64
heartRate          float64
glucose            float64
TenYearCHD           int64
dtype: object

In [4]:
# TIỀN XỬ LÝ DỮ LIỆU
# hiện thị số giá trị null của từng trường
df.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [5]:
# xóa các giá trị null đó đi
final_df = df.dropna()

In [6]:
# xem số lượng của mỗi lớp (1/0) để quyết định số dữ liệu lấy mẫu
final_df["TenYearCHD"].value_counts()


0    3101
1     557
Name: TenYearCHD, dtype: int64

In [7]:
# chia dữ liệu thành các lớp riêng biệt
df_NoCHD = final_df[final_df['TenYearCHD']==0]
df_CHD = final_df[final_df['TenYearCHD']==1] 
# tăng/giảm các mẫu trong mỗi lớp
df_NoCHD_downsampled = resample(df_NoCHD, replace=True, n_samples=1800, random_state=123)
df_CHD_upsampled = resample(df_CHD, replace=True, n_samples=1800, random_state=123)


In [8]:
df_CHD_upsampled

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
3906,0,47,3.0,0,0.0,0.0,0,0,0,230.0,137.0,79.0,27.13,75.0,76.0,1
2803,1,62,1.0,0,0.0,0.0,0,1,0,191.0,156.0,91.0,31.20,68.0,75.0,1
2930,0,57,1.0,1,43.0,0.0,0,1,0,283.0,207.5,118.0,38.61,100.0,83.0,1
2469,1,48,4.0,1,20.0,0.0,0,1,0,259.0,135.0,90.0,20.72,102.0,81.0,1
656,1,51,1.0,1,20.0,0.0,0,0,0,243.0,130.0,86.5,29.86,85.0,74.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,1,65,2.0,0,0.0,0.0,0,0,1,238.0,122.0,81.0,23.95,67.0,150.0,1
2343,1,37,4.0,0,0.0,0.0,0,0,0,266.0,110.0,72.5,26.09,77.0,73.0,1
3077,0,67,1.0,0,0.0,0.0,0,0,0,223.0,111.0,73.0,27.89,90.0,63.0,1
2460,1,59,2.0,0,0.0,0.0,0,0,0,299.0,120.5,78.0,25.45,60.0,100.0,1


In [9]:
# nối các lớp lại với nhau
df2 = pd.concat([df_NoCHD_downsampled, df_CHD_upsampled])
df2['TenYearCHD'].value_counts()

0    1800
1    1800
Name: TenYearCHD, dtype: int64

In [10]:
# bỏ trường biến phụ thuộc, tách biến phụ thuộc ra biến độc lập
X = df2.iloc[:,:-1]
# StandardScaler() dùng để quy đổi tỷ lệ các giá trị khác nhau để so sánh
sc = StandardScaler()
X = sc.fit_transform(X)
X = sc.transform(X)
#xáo trộn các hàng giúp tăng độ ngẫu nhiên
df2 = df2.sample(frac=1).reset_index(drop=True)
# cột đầu ra: biến phụ thuộc
Y  = df2['TenYearCHD']


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [11]:
# tách dữ liệu thành tập xây dựng và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
print((y_train == 1).sum())
print((y_train == 0).sum())

1260
1260


In [12]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.values.astype(np.float32))
y_test = torch.from_numpy(y_test.values.astype(np.float32))

In [13]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [14]:
# đưa tensor đầu ra về dạng vector cột để đảm bảo tính nhất quán về kích thước và kiểu dữ liệu
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [15]:
# xây dựng mô hình
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features,1)
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [16]:
model = LogisticRegression(X_train.shape[1])

In [17]:
num_epochs = 500                                        
# Traning the model for large number of epochs to see better results  
learning_rate = 0.0001                               
criterion = nn.BCELoss()                                
# We are working on lgistic regression so using Binary Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)      
# Using ADAM optimizer to find local minima  

In [18]:
for epoch in range(num_epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)             
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1) % 20 == 0:                                         
        # printing loss values on every 10 epochs to keep track
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 20, loss = 1.2740
epoch: 40, loss = 1.2719
epoch: 60, loss = 1.2701
epoch: 80, loss = 1.2686
epoch: 100, loss = 1.2672
epoch: 120, loss = 1.2659
epoch: 140, loss = 1.2648
epoch: 160, loss = 1.2638
epoch: 180, loss = 1.2628
epoch: 200, loss = 1.2619
epoch: 220, loss = 1.2611
epoch: 240, loss = 1.2603
epoch: 260, loss = 1.2595
epoch: 280, loss = 1.2587
epoch: 300, loss = 1.2580
epoch: 320, loss = 1.2573
epoch: 340, loss = 1.2566
epoch: 360, loss = 1.2559
epoch: 380, loss = 1.2553
epoch: 400, loss = 1.2546
epoch: 420, loss = 1.2540
epoch: 440, loss = 1.2533
epoch: 460, loss = 1.2527
epoch: 480, loss = 1.2520
epoch: 500, loss = 1.2514


In [19]:

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.5065


In [20]:
from sklearn.metrics import classification_report
# báo cáo phân loại: tổng hợp các chỉ số đánh giá hiệu suất của mô hình
print(classification_report(y_test, y_predicted_cls))

              precision    recall  f1-score   support

         0.0       0.51      0.57      0.54       540
         1.0       0.51      0.44      0.47       540

    accuracy                           0.51      1080
   macro avg       0.51      0.51      0.50      1080
weighted avg       0.51      0.51      0.50      1080



In [21]:
from sklearn.metrics import confusion_matrix
# ma tran nham lan
confusion_matrix = confusion_matrix(y_test, y_predicted_cls)
print(confusion_matrix)

[[307 233]
 [300 240]]


In [22]:
# Chuyển dữ liệu mới thành tensor
male = 0
age = 61
education = 3
currentSmoker= 1
cigsPerDay= 30
BPMeds= 0
prevalentStroke= 0
prevalentHyp= 1
diabetes = 0
totChol = 225
sysBP= 150
diaBP = 95
BMIm = 28.58
heartRate  = 65
glucose = 103


new_data = [male ,age,education ,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes ,totChol ,sysBP,diaBP ,BMIm ,heartRate  ,glucose ]
new_data_tensor = torch.tensor(new_data, dtype=torch.float32).unsqueeze(0)

# Dự đoán đầu ra cho dữ liệu mới
with torch.no_grad():
    output = model(new_data_tensor)
    predicted = output.round()
    print(predicted.item())

0.0
